In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# reading image from path (BGR format)
img=cv2.imread('Broadway_tower.jpg')

In [3]:
# Scharr deriviation
def energy_image(image):
    # turn color image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # calculating maximum of HOG(Image)
    hog = cv2.HOGDescriptor()
    h = hog.compute(gray).max()
    # derivation of image in x and y direction
    scharr_x = cv2.Scharr(gray,-1,1,0)
    scharr_y = cv2.Scharr(gray,-1,0,1)
    abs_scharr_x = cv2.convertScaleAbs(scharr_x)
    abs_scharr_y = cv2.convertScaleAbs(scharr_y)
    # calculate energy function (e_HOG)
    energy_map = (abs_scharr_x +  abs_scharr_y)/h
    return energy_map

In [4]:
# this function calculate cost matrix of image ,and also all seams path
def getMaps(image,energy_map=0):
    # extra condition for object removal
    if type(energy_map)==int:
        energy_map = energy_image(image)
    rows, columns, = energy_map.shape[:2]
    cost_matrix = energy_map.copy()
    # create a copy of energy map to calculate seams
    goback = np.zeros_like(cost_matrix, dtype = np.int)
    for i in range(1, rows):
        for j in range(0, columns):
            
            if j == 0:
                # create seams
                min_index = np.argmin(cost_matrix[i - 1, j : j + 2])
                # create seam path
                goback[i, j] = min_index + j
                # create energy at each pixel
                min_energy = cost_matrix[i - 1, min_index + j]
                
            else:
                min_index = np.argmin(cost_matrix[i - 1, j - 1 : j + 2])
                goback[i, j] = min_index + j -1
                min_energy = cost_matrix[i - 1, min_index + j - 1]

            cost_matrix[i, j] += min_energy

    return cost_matrix, goback

In [5]:
# finding minimum seam of an image
def find_seam(image,energy_map=0):
    row , col=image.shape[:2]
    cost_matrix,goback=getMaps(image,energy_map)
    # finding minimum value in last row of cost matrix
    min_index=np.argmin(cost_matrix[-1])
    min_seam=[min_index]
    j=min_index
    # finding seam with minimum energy using goback matrix
    for i in range(row-1,0,-1):
        min_seam.append(goback[i,j])
        j=goback[i,j]
    # change the order of minimum seam indices from up to bottom
    min_seam.reverse()
    return min_seam

In [6]:
def draw_seam(image,seam):
    # draw a seam on image with color Red
    copy_image=image.copy()
    for i in range(len(seam)):
            copy_image[i,seam[i]]=np.array([0,0,255]).astype(np.uint8)
    return copy_image

In [7]:
def delete_seam(image, seam):
    rows, cols = image.shape[:2]
    #remove seam from image and adjust all other columns
    for row in range(rows):
        for col in range(int(seam[row]), cols-1):
            #shift columns
            image[row, col] = image[row, col+1]
    # remove the last column
    image = image[:, 0:cols-1]
    return image

In [8]:
def insert_seam(image, seam):
    rows, cols, dim = image.shape
    new_img=np.zeros((rows,cols+1,dim),dtype= np.uint8)
    # copy images in new_image except minimum seam
    for i in range(rows):
        j=seam[i]
        if j==0:
            new_img[i,j+2:]=image[i,j+1:].astype(np.uint8)
            # calculate the inserting pixels by averaging the seam pixel and it's two neighbor
            insertion=np.average(image[i,j:j+2],axis=0).astype(np.uint8)
        else:
            new_img[i,:j]=image[i,:j].astype(np.uint8)
            new_img[i,j+2:]=image[i,j+1:].astype(np.uint8)
            insertion=np.average(image[i,j-1:j+2],axis=0).astype(np.uint8)
        
        new_img[i,j]=insertion
        new_img[i,j+1]=insertion
    
    return new_img

In [9]:
def seam_carving(img,iterates,direction,change):
    # direction : horizontal or vertoical
    # change : reduce(remove seam) or enlarge(insert seam)
    # add changes in a list
    video=[]
    # adding original image
    video.append(img)
    if direction == 'horizontal':
        # implement all methods on transpose of image to get horizontal changes
        img_t=cv2.transpose(img)
        for iterate in range(iterates):
            min_seam=find_seam(img_t)
            video.append(cv2.transpose(draw_seam(img_t,min_seam)))
            if change=='reduce':
                img_t=delete_seam(img_t,min_seam)
            elif change=='enlarge':
                img_t=insert_seam(img_t,min_seam)
            video.append(cv2.transpose(img_t))
    
    elif direction == 'vertical':
        for iterate in range(iterates):
            min_seam=find_seam(img)
            video.append(draw_seam(img,min_seam))
            if change=='reduce':
                img=delete_seam(img,min_seam)
            elif change=='enlarge':
                img=insert_seam(img,min_seam)
            video.append(img)
    
    return video

In [10]:
def video(name,result,change):
    #save a video of changes 
    if change=='reduce':
        h, w= result[0].shape[:2]
        size = (w,h)
        video = cv2.VideoWriter(str(name)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1.0, size)
        # resize images 
        for image in result:
            if image.shape[0]!=h :
                resized=cv2.copyMakeBorder(image.copy(),np.abs(h-image.shape[0]),0,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
            elif image.shape[1]!=w:
                resized=cv2.copyMakeBorder(image.copy(),0,0,0,np.abs(w-image.shape[1]),cv2.BORDER_CONSTANT,value=[0,0,0])
            else:
                resized=image
            video.write(resized)
            
    elif change =='enlarge':
        result.reverse()
        temp=[]
        h, w= result[0].shape[:2]
        size = (w,h)
        video = cv2.VideoWriter('video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1.0, size)
        for image in result:
            if image.shape[0]!=h :
                resized=cv2.copyMakeBorder(image.copy(),np.abs(h-image.shape[0]),0,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
                temp.append(resized)
            elif image.shape[1]!=w:
                resized=cv2.copyMakeBorder(image.copy(),0,0,0,np.abs(w-image.shape[1]),cv2.BORDER_CONSTANT,value=[0,0,0])
                temp.append(resized)
            else:
                resized=image
                temp.append(resized)
                
        temp.reverse()
        for image in temp:
            video.write(image)
    
    
    cv2.destroyAllWindows()
    video.release()

In [ ]:
#example
result=seam_carving(img,30,'vertical','reduce')

In [ ]:
video('seamCarving',result,'reduce')

## object removing sample

In [11]:
from skimage import  draw

In [12]:
img=cv2.imread('Broadway_tower.jpg')

In [13]:
image=img.copy()
# person position
# 90 * 30 window showing the object
poly=[(715,114),(715,144),(805,144),(805,114)]
p1 = np.array([p[0] for p in poly])
p2 = np.array([p[1] for p in poly])
rr, cc = draw.polygon(p1, p2)
color = np.array([0, 1, 0])
image[rr, cc, :] = image[rr, cc, :]*0.5 + color*0.5

In [14]:
energy_map=energy_image(img)
# reduce energy in removal window
energy_map[rr, cc] -= 100000

In [15]:
deleted_seams=[]
obj_removal=[]
# displaying removal object
obj_removal.append(image)

In [16]:
# window is 90*30
# implementing seam carving for 30 times to remove object in vertical direction
for iterate in range(30) :
    s=find_seam(img,energy_map)
    deleted_seams.append(s)
    obj_removal.append(draw_seam(img,s))
    img=delete_seam(img,s)
    obj_removal.append(img)
    # remove the previous seam from energy map
    energy_map=delete_seam(energy_map,s)

In [17]:
video('objectRemoval',obj_removal,'reduce')

In [18]:
# reordering seams from first to last seams that's been removed
deleted_seams.reverse()

In [19]:
increase=[]

In [ ]:
for s in deleted_seams:
    increase.append(draw_seam(img,s))
    img=insert_seam(img,s)
    increase.append(img)

In [ ]:
video('lastResult',increase,'enlarge')